In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession

spark = (SparkSession.builder.
         appName("working with tabular data").
         getOrCreate())

In [4]:
import os
import pyspark.sql.functions as F

In [5]:
DIRECTORY = r"C:\Users\oluwa\Documents\broadcast_logs"

In [ ]:
#%%bash

#!ls DIRECTORY
files = os.listdir(DIRECTORY)

for f in files:
    print(f)
print("-"*100)
for f in files:
    print(os.path.join(DIRECTORY, f))

In [6]:
logs = spark.read.csv(
    os.path.join(DIRECTORY, "BroadcastLogs_2018_Q3_M8.csv"),
    sep = "|",
    header = True,
    inferSchema = True,
    timestampFormat = "yyyy-MM-dd",
)

In [ ]:
logs.printSchema()

In [ ]:
logs.select("BroadcastLogID", "LogServiceID", "LogDate").show(5, False)

In [7]:
logs = logs.select(*[x for x in logs.columns if x not in ["BroadcastLogID", "LogServiceID", "LogDate"]])

In [ ]:
import pandas as pd

"Language1" in logs.columns

logs.columns

In [ ]:
logs.select('BroadcastLogID',
 'LogServiceID',
 'LogDate').show(5, truncate=False)

In [8]:
#exclude 'BroadcastLogID' and 'SequenceNO' columns

logs_copy = logs.select(*[x for x in logs.columns if x not in ['BroadcastLogID', 'SequenceNO', 'LogServiceID']])

In [ ]:
logs_copy.columns

In [ ]:
logs_copy.select(F.col('LogDate')).show(5, False)

In [ ]:
logs_copy.select(F.substring(F.col('LogDate'), 1, 4)).distinct().show(5)

In [ ]:
logs_copy.select(
    F.col('Duration').alias("full_duration")).show(5, False)

In [ ]:
logs_copy.select(
    F.col('Duration').alias('full'),
    F.col('Duration').substr(1,2).cast('int').alias('hour'),
    F.col('Duration').substr(4,2).cast('int').alias('min'),
    F.col('Duration').substr(7,2).cast('int').alias('sec'),
    (F.col('Duration').substr(1,2).cast('int') * 60 * 60 +
    F.col('Duration').substr(4,2).cast('int') * 60 +
    F.col('Duration').substr(7,2).cast('int')).alias('duration_secs')
).distinct().show(5, False)

In [ ]:
logs_copy = logs.withColumn('duration_secs',
F.col('Duration').substr(1,2).cast('int') * 60 * 60 +
F.col('Duration').substr(4,2).cast('int') * 60 +
F.col('Duration').substr(7,2).cast('int'))

In [ ]:
logs_copy.printSchema()

In [ ]:
logs_copy.select(F.col('Duration'), F.col('duration_secs')).show(5, False)

In [ ]:
logs_copy = logs_copy.select(*[x.lower() for x in logs_copy.columns])

In [ ]:
logs_copy.printSchema()

In [ ]:
logs_copy.columns

In [ ]:
for w in logs_copy.columns:
    logs_copy.describe(w).show()

In [ ]:
logs_copy.select(F.col('LogDate')).show(5)

In [ ]:
logs_copy = logs.withColumn('duration_secs',
F.col('Duration').substr(1,2).cast('int') * 60 * 60 +
F.col('Duration').substr(4,2).cast('int') * 60 +
F.col('Duration').substr(7,2).cast('int'))

In [9]:
logs_copy = logs_copy.withColumn('seconds_duration',
F.col('Duration').substr(1,2).cast('int')*60*60 +
F.col('Duration').substr(4,2).cast('int')*60 +
F.col('duration').substr(7,2).cast('int'))

In [ ]:
logs_copy.select(F.col('seconds_duration'), F.col('Duration').substr(1,8)).show(5, False)

In [10]:
# rename all column headers to lowercase with toDF

logs_copy = logs_copy.toDF(*[x.lower() for x in logs_copy.columns])

In [11]:
logs_copy.printSchema

<bound method DataFrame.printSchema of DataFrame[audiencetargetageid: int, audiencetargetethnicid: int, categoryid: int, closedcaptionid: int, countryoforiginid: int, dubdramacreditid: int, ethnicprogramid: int, productionsourceid: int, programclassid: int, filmclassificationid: int, exhibitionid: int, duration: string, endtime: string, logentrydate: timestamp, productionno: string, programtitle: string, starttime: string, subtitle: string, networkaffiliationid: int, specialattentionid: int, broadcastoriginpointid: int, compositionid: int, producer1: string, producer2: string, language1: int, language2: int, seconds_duration: int]>